In [1]:
pip install -U scikit-learn 

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/Openclassrooms/P7 scoring/git_collab_P7/git_collab_P7')

import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE 
import sklearn


import joblib

In [2]:
from My_Fonctions.fonctions import *

In [3]:
data = "/content/drive/MyDrive/Openclassrooms/P7 scoring/data/"

## pre processing du data set

Fonction réalisé à partir du kernel Kaggle : https://www.kaggle.com/jsaguiar/lightgbm-with-simple-features

In [ ]:
df_data = preproc(data,num_row=None)

Train samples: 307511
Bureau df shape: (305811, 116)
Process bureau and bureau_balance - done in 32s
Previous applications df shape: (338857, 249)
Process previous_applications - done in 28s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 14s
Installments payments df shape: (339587, 26)
Process installments payments - done in 39s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 22s


Sauvegarde du data set

In [ ]:
joblib.dump(df_data, data + 'df_data.joblib')

['/content/drive/MyDrive/Openclassrooms/P7 scoring/data/df_data.joblib']

In [ ]:
df_data.shape

(356251, 798)

In [ ]:
df_data['TARGET'].value_counts()

0.0    282682
1.0     24825
Name: TARGET, dtype: int64

In [ ]:
df_data['TARGET'].isna().sum() 

48744

In [26]:
df_data.dtypes

index                                 int64
SK_ID_CURR                            int64
TARGET                              float64
CODE_GENDER                           int64
FLAG_OWN_CAR                          int64
                                     ...   
CC_NAME_CONTRACT_STATUS_nan_MAX     float64
CC_NAME_CONTRACT_STATUS_nan_MEAN    float64
CC_NAME_CONTRACT_STATUS_nan_SUM     float64
CC_NAME_CONTRACT_STATUS_nan_VAR     float64
CC_COUNT                            float64
Length: 798, dtype: object

# restauration du dataset

In [4]:
df_data = joblib.load(data + 'df_data.joblib')

Séparation du data set train / test

In [5]:
df_data_train = df_data[df_data["TARGET"].notnull() ]

In [6]:
df_data_test =  df_data[df_data["TARGET"].isnull() ]

# Péparation du data set train

Le jeux de donnée est désiquilibre.
Nous utilison l'option de stratify lors de la selection des du train pour conserver le meme désequilibre

In [7]:
train, test = train_test_split(df_data_train, test_size=0.2, stratify=df_data_train['TARGET']) 

In [8]:
df_data['TARGET'].value_counts()

0.0    282682
1.0     24825
Name: TARGET, dtype: int64

In [9]:
train['TARGET'].value_counts()

0.0    226145
1.0     19860
Name: TARGET, dtype: int64

In [10]:
test['TARGET'].value_counts()

0.0    56537
1.0     4965
Name: TARGET, dtype: int64

Le ration de 10% est conservé entre les target 1 et 0

Reduction du jeux de donnée qui est trop important pour tester plusieur modele et methode de gestion des déséquilibres de données

In [11]:
y_train = train["TARGET"]
X_train = train.drop(["TARGET"],axis=1)
y_test = test["TARGET"]
X_test = test.drop(["TARGET"],axis=1)

In [24]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.9, random_state=0)
sss.get_n_splits(X_train, y_train)

for train_index, test_index in sss.split(X_train, y_train):
    
    #selection d'un échantillon aléatoire stratifié pour respecter les proportions
    
    X_sample = X_train.iloc[train_index,:] 
    y_sample = y_train.iloc[train_index]

# fonction de cout utilisé

La librairie imbleanr permet d'afficher des metrics specifique au dataset désequilibré tel que le score F1 et la moyenne geometrique

# test de modele

In [22]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [27]:
X_sample.replace([np.inf, -np.inf], np.nan, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [32]:
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [28]:
X_sample_nan = imp.fit_transform(X_sample)

In [33]:
X_test_nan= imp.transform(X_test)

In [29]:
lr = LogisticRegression()


In [30]:
lr.fit(X_sample_nan, y_sample)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [35]:
y_pred = lr.predict(X_test_nan)

In [36]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.92      1.00      0.00      0.96      0.05      0.00     56537
        1.0       0.13      0.00      1.00      0.01      0.05      0.00      4965

avg / total       0.86      0.92      0.08      0.88      0.05      0.00     61502

